In [ ]:
import warnings
warnings.simplefilter(action='ignore', category=FutureWarning)
import tensorflow_hub as hub
embed = hub.load("https://tfhub.dev/google/universal-sentence-encoder/4")

In [ ]:
import json
import matplotlib.pyplot as plt
import pandas as pd
from sklearn.datasets import make_blobs
from sklearn.cluster import Birch
import plotly.express as px

In [ ]:
import json
import re, string
import pandas as pd
import numpy as np
from tqdm.auto import tqdm, trange

from sklearn.model_selection import train_test_split
from sklearn.pipeline import Pipeline
from sklearn.feature_extraction.text import TfidfTransformer
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score
from sklearn.metrics import classification_report
!pip install transformers
from transformers import AutoTokenizer, AutoModel
import torch

import tensorflow_hub as hub
import numpy as np
from sklearn.cluster import AgglomerativeClustering
import matplotlib.pyplot as plt
from sklearn.decomposition import PCA


     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.6/7.6 MB 56.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 268.8/268.8 kB 28.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.8/7.8 MB 89.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.3/1.3 MB 66.7 MB/s eta 0:00:00


In [ ]:
import os

def build_data_frame_from_jsons():
  df = pd.DataFrame(columns=['answer'])

  directory = 'example'
  for filename in os.scandir(directory):
      if filename.is_file():
          with open(filename.path) as f:
              decoded = f.read().encode().decode('utf-8-sig')
              text = json.loads(decoded)
              answers = text["answers"] # context cluster

              answers_texts = []
              for i in answers:
                answers_texts.append(i['answer'])

              df1 = pd.DataFrame(columns=['answer'])
              df1.loc[:, 'answer'] = answers_texts

              df = pd.concat([df, df1], ignore_index = True)
              df.reset_index()

  df = df.replace(r'^s*$', float('NaN'), regex = True)
  df = df.dropna(subset=['answer'])

  df = df.reset_index()
  df = df.drop(['index'], axis=1)

  return df

In [ ]:
df_1 = build_data_frame_from_jsons()
'''
embeds = embed(df_1['answer'])
embeds'''

"\nembeds = embed(df_1['answer'])\nembeds"

In [ ]:
'''X = np.array(embeds)

model = Birch(n_clusters=2)
model.fit(X)

pred = model.predict(X)
pred2 = np.unique(pred)

labels = model.predict(embeds)

plt.scatter(embeds[:, 0], embeds[:, 1], c=labels, cmap='rainbow')
plt.show()'''

"X = np.array(embeds)\n\nmodel = Birch(n_clusters=2)\nmodel.fit(X)\n\npred = model.predict(X)\npred2 = np.unique(pred)\n\nlabels = model.predict(embeds)\n\nplt.scatter(embeds[:, 0], embeds[:, 1], c=labels, cmap='rainbow')\nplt.show()"

In [ ]:
'''# Кластеризация с помощью AgglomerativeClustering с неопределенным количеством кластеров
cluster = AgglomerativeClustering(n_clusters=None, distance_threshold=1.25)
labels = cluster.fit_predict(embeds)

# Снижение размерности эмбеддингов для визуализации
pca = PCA(n_components=2)
embeddings_2d = pca.fit_transform(embeds)

# Визуализация результатов кластеризации
plt.scatter(embeddings_2d[:, 0], embeddings_2d[:, 1], c=labels)
plt.title("Кластеризация предложений")
plt.show()
'''


'# Кластеризация с помощью AgglomerativeClustering с неопределенным количеством кластеров\ncluster = AgglomerativeClustering(n_clusters=None, distance_threshold=1.25)\nlabels = cluster.fit_predict(embeds)\n\n# Снижение размерности эмбеддингов для визуализации\npca = PCA(n_components=2)\nembeddings_2d = pca.fit_transform(embeds)\n\n# Визуализация результатов кластеризации\nplt.scatter(embeddings_2d[:, 0], embeddings_2d[:, 1], c=labels)\nplt.title("Кластеризация предложений")\nplt.show()\n'

In [ ]:
texts = df_1['answer']

In [ ]:


tokenizer = AutoTokenizer.from_pretrained("cointegrated/LaBSE-en-ru")
model = AutoModel.from_pretrained("cointegrated/LaBSE-en-ru")



In [ ]:
def get_embendLabSE(texts):
  embeddings_list = []
  for text in texts:
      encoded_input = tokenizer(text, padding=True, truncation=True, max_length=64, return_tensors='pt')
      with torch.no_grad():
          model_output = model(**encoded_input)
      embedding = model_output.pooler_output
      embeddings_list.append((embedding)[0].numpy())
  embeddings = np.asarray(embeddings_list)
  return(embeddings)

In [ ]:
modelRUB = AutoModel.from_pretrained("DeepPavlov/distilrubert-small-cased-conversational")
tokenizerRUB = AutoTokenizer.from_pretrained("DeepPavlov/distilrubert-small-cased-conversational")

In [ ]:
from tqdm import tqdm

def get_embend(sents):
  embends = []
  for sent in tqdm(sents):
    tokens = tokenizerRUB(sent, padding='max_length', max_length=512, truncation=True, return_tensors="pt")
    embendings = modelRUB(input_ids=tokens["input_ids"], attention_mask=tokens["attention_mask"], return_dict=True)
    embends.append(embendings.last_hidden_state[0][0].tolist())
  return embends

In [ ]:


embeddingsRuB = get_embend(texts)

100%|██████████| 57/57 [00:49<00:00,  1.14it/s]


In [ ]:
clusteringRuB = AgglomerativeClustering(n_clusters=None, distance_threshold=0.2, metric = 'cosine', linkage='average').fit(embeddingsRuB)

In [ ]:
pca = PCA(n_components=2)
emb_2dRub = pd.DataFrame(pca.fit_transform(embeddingsRuB), columns=['x1', 'x2'])
emb_2dRub['label'] = clusteringRuB.labels_
emb_2dRub['label'].nunique()

29

In [ ]:
df_1.loc[:, 'labels'] = emb_2dRub['label']
df_1[df_1.labels == 6]

,answer,labels
2,классно,6
4,супер,6
7,супер,6
13,шикарно,6
14,всё супер,6
16,класно,6
17,великолепно,6
18,замечательно,6
31,прекрасно,6
33,хорошо,6


In [ ]:
fig = px.scatter(emb_2dRub, x='x1', y='x2', color='label', width=800, height=600)
fig.show()

In [ ]:
embeddingsLABSE = get_embendLabSE(texts)

In [ ]:
clusteringLABSE = AgglomerativeClustering(n_clusters=None, distance_threshold=0.3, metric = 'cosine', linkage='average').fit(embeddingsLABSE)

In [ ]:
emb_2dLABSE = pd.DataFrame(pca.fit_transform(embeddingsLABSE), columns=['x1', 'x2'])
emb_2dLABSE['label'] = clusteringLABSE.labels_
emb_2dLABSE['label'].nunique()

43

In [ ]:
df_1.loc[:, 'labels'] = emb_2dLABSE['label']
df_1[df_1.labels == 6]

,answer,labels
52,скайп,6


In [ ]:
fig = px.scatter(emb_2dLABSE, x='x1', y='x2', color='label', width=800, height=600)
fig.show()

In [ ]:
from sklearn.metrics import silhouette_score


X = embeddingsRuB

# Создадим объект KMeans и выполним кластеризацию

labels = clusteringRuB.labels_

# Вычислим индекс Силуэтта
silhouette_avg = silhouette_score(X, labels)
print("Средний индекс Силуэтта:", silhouette_avg)


Средний индекс Силуэтта: 0.09754150745786153


In [ ]:
import pickle


model = clusteringRuB

filename = 'model_v1.pk'
with open(filename, 'wb') as file:
  pickle.dump(clusteringRuB, file)

In [ ]:
texts[emb_2dRub['label']]
texts_arr = texts.values

In [ ]:
'''def show_examples(cluster, n):
    for i in range(n):
        print(i, texts[emb_2dRub.label == cluster][i].split('.')[0])
show_examples(cluster=0, n=3)'''

"def show_examples(cluster, n):\n    for i in range(n):\n        print(i, texts[emb_2dRub.label == cluster][i].split('.')[0])\nshow_examples(cluster=0, n=3)"

In [ ]:
docs_df = pd.DataFrame(texts_arr, columns=["Doc"])
docs_df['Topic'] = clusteringRuB.labels_
docs_df['Doc_ID'] = range(len(docs_df))

docs_per_topic = docs_df.groupby(['Topic'], as_index = False).agg({'Doc': ' '.join})
docs_per_topic


,Topic,Doc
0,0,продуктивно дружелюбно бодро
1,1,удаленка работа проверка
2,2,город россия москва авто
3,3,отлчино душ
4,4,снова на работу
5,5,гуд отвратительно приемлемо норм
6,6,классно супер супер шикарно всё супер класно в...
7,7,на пятерочку
8,8,это личное
9,9,не хочу отвечать


In [ ]:
from transformers import AutoModelForSeq2SeqLM, T5TokenizerFast

In [ ]:
class TextAnalyzer:
    def __init__(self, texts):
        self.texts = texts

    def process_texts(self):
        docs_df = self.texts

        docs_per_topic = docs_df.groupby(['Topic'], as_index=False).agg({'Doc': ' '.join})

        tf_idf, count = self.c_tf_idf(docs_per_topic.Doc.values, m=len(docs_df))

        return tf_idf, count

    @staticmethod
    def c_tf_idf(documents, m, ngram_range=(1, 1)):
        count = CountVectorizer(ngram_range=ngram_range, stop_words="english").fit(documents)
        t = count.transform(documents).toarray()
        w = t.sum(axis=1)
        tf = np.divide(t.T, w)
        sum_t = t.sum(axis=0)
        idf = np.log(np.divide(m, sum_t)).reshape(-1, 1)
        tf_idf = np.multiply(tf, idf)

        return tf_idf, count

    @staticmethod
    def extract_top_n_words_per_topic(tf_idf, count, docs_per_topic, n=7):
        words = count.get_feature_names_out()
        labels = list(docs_per_topic.Topic)
        tf_idf_transposed = tf_idf.T
        indices = tf_idf_transposed.argsort()[:, -n:]
        top_n_words = {label: [words[j] for j in indices[i]][::-1] for i, label in enumerate(labels)}
        return top_n_words

    @staticmethod
    def process_sum_list(sum_list):
        MODEL_NAME = 'UrukHan/t5-russian-spell'
        MAX_INPUT = 256

        tokenizer = T5TokenizerFast.from_pretrained(MODEL_NAME)
        modelT5 = AutoModelForSeq2SeqLM.from_pretrained(MODEL_NAME)

        result = {}
        n = 0
        for sublist in sum_list:
            input_sequences = sublist[:3]
            input_sequences = ' '.join(input_sequences)

            task_prefix = "Spell correct: "
            if type(input_sequences) != list:
                input_sequences = [input_sequences]

            encoded = tokenizer(
                [task_prefix + sequence for sequence in input_sequences],
                padding="longest",
                max_length=MAX_INPUT,
                truncation=True,
                return_tensors="pt",
            )

            device = torch.device('cpu')
            predicts = modelT5.generate(**encoded.to(device))
            sum1 = tokenizer.batch_decode(predicts, skip_special_tokens=True)

            result[n] = sum1
            n += 1

        return result
    @staticmethod
    def remove_duplicates(dictionary):
        for key, value in dictionary.items():
            dictionary[key] = list(set(value))
        return dictionary

# docs_df - DataFrame с столбцами "Doc" и "Topic"
analyzer = TextAnalyzer(docs_df)
tf_idf, count = analyzer.process_texts()

top_words = analyzer.extract_top_n_words_per_topic(tf_idf, count, docs_per_topic, n=2)
sum_list = list(top_words.values())  # Ваш список sum_list
results = analyzer.process_sum_list(sum_list)
sum_final = analyzer.remove_duplicates(results)

print(results)



{0: ['продуктивно, дружелюбно.'], 1: ['Работа, проверка'], 2: ['. Москва. Авто.'], 3: ['Душ Отличино.'], 4: ['Работу снова.'], 5: ['норм приемлемо.'], 6: ['Супер, великолепно.'], 7: ['Пятичку на.'], 8: ['Личное это.'], 9: ['Не хочу.'], 10: ['«Харашо это.'], 11: ['Задачи, интерес.'], 12: ['Завтрак молоком.'], 13: ['Amazing: Это...'], 14: ['Аду Гори'], 15: ['Сообщения, задача.'], 16: ['123 начинается'], 17: ['Все лес.'], 18: ['Зачем знать.'], 19: ['Скайп это.'], 20: ['nIou это.'], 21: ['Зарядка это.'], 22: ['Расслабляющая пробежка.'], 23: ['За, конечно.'], 24: ['«Утро началось.'], 25: ['В жизни разговора.'], 26: ['Турник это.'], 27: ['Неделю нет.'], 28: ['Пробежка это.']}


In [ ]:
top_words

{0: ['продуктивно', 'дружелюбно'],
 1: ['работа', 'проверка'],
 2: ['москва', 'авто'],
 3: ['душ', 'отлчино'],
 4: ['работу', 'снова'],
 5: ['норм', 'приемлемо'],
 6: ['супер', 'великолепно'],
 7: ['пятерочку', 'на'],
 8: ['личное', 'это'],
 9: ['не', 'хочу'],
 10: ['харашо', 'это'],
 11: ['задачи', 'интерес'],
 12: ['завтрак', 'молоком'],
 13: ['amazing', 'это'],
 14: ['аду', 'сгори'],
 15: ['сообщения', 'задача'],
 16: ['123', 'начинается'],
 17: ['все', 'лес'],
 18: ['зачем', 'знать'],
 19: ['скайп', 'это'],
 20: ['niou', 'это'],
 21: ['зарядка', 'это'],
 22: ['расслабляющая', 'пробежки'],
 23: ['за', 'конечно'],
 24: ['утро', 'началось'],
 25: ['жизни', 'разговора'],
 26: ['турник', 'это'],
 27: ['неделю', 'нет'],
 28: ['пробежка', 'это']}

In [ ]:
# Импортируем библиотеки
from transformers import AutoModelForSeq2SeqLM, T5TokenizerFast

In [ ]:
# Зададим название выбронной модели из хаба
MODEL_NAME = 'UrukHan/t5-russian-spell'
MAX_INPUT = 256

In [ ]:
# Загрузка модели и токенизатора
tokenizer = T5TokenizerFast.from_pretrained(MODEL_NAME)
model = AutoModelForSeq2SeqLM.from_pretrained(MODEL_NAME)

In [ ]:
# Входные данные (можно массив фраз или текст)
input_sequences = [word for word in top_n_words[1][:2]]
input_sequences = ' '.join(input_sequences)
# можно использовать одиночные фразы:
# input_sequences = 'сеглдыя хорош ден'
input_sequences

NameError: ignored

In [ ]:
# Токенизирование данных
task_prefix = "Spell correct: "
if type(input_sequences) != list: input_sequences = [input_sequences]
encoded = tokenizer(
  [task_prefix + sequence for sequence in input_sequences],
  padding="longest",
  max_length=MAX_INPUT,
  truncation=True,
  return_tensors="pt",
)

In [ ]:
import torch


device = torch.device('cpu')

In [ ]:
predicts = model.generate(**encoded.to(device))

In [ ]:
# Декодируем данные
sum2 = tokenizer.batch_decode(predicts, skip_special_tokens=True)
sum2